In [105]:
import sys
import os
import time
import datetime as dt

import numpy as np
import scipy.ndimage
import pandas as pd
import matplotlib.pyplot as plt

from osgeo import gdal
from osgeo import osr
from osgeo.gdalconst import *
driver = gdal.GetDriverByName('GTiff')
driver.Register()



2

In [136]:
dir_path = os.path.realpath('./')
cwd = os.getcwd()

watershed_name = "Sleepers River"
watershed_foldername = "sleepers"
watershed_nick = "sleepers"
working_dir = (os.path.abspath(os.path.join(dir_path,'..')) + "/data/" + 
               watershed_foldername + os.path.sep)
n_days = 20000
run_number = 1
run_folder = watershed_nick + '_' + str(n_days) + '__' + str(run_number) + os.path.sep


dem = working_dir + watershed_nick + "_dem.tif"
elevations_ds = gdal.Open(dem, GA_ReadOnly)
band = elevations_ds.GetRasterBand(1)
cols, rows = elevations_ds.RasterXSize, elevations_ds.RasterYSize
elevationValues = band.ReadAsArray(0, 0, cols, rows)
geotransform = elevations_ds.GetGeoTransform()
resol = np.round(geotransform[1])
watershed_cells = np.where(elevationValues>0.0, 1, 0)
A = np.sum(watershed_cells) * resol * resol

#parameters for sand  
Ks = 20  #micrometers per second
Ks = Ks * 3600 / 1000000  #meters per hourd
psi_b = 0.1  #bubbling pressure / capillary head, for sand
fi = 0.45  #porosity
Sy = 0.22  #specific yield
b = 0.00010
n = 0.6

D = 2
#ET = 3/24
ET = {'May':3/24, 'June':3.5/24, 'July':3.5/24,
     'August':3.5/24, 'September':3/24, 'October':3/24}
maxStorage = A * D * fi




relationship_filename = run_folder+watershed_nick+"_{}_{}_ARatio_satFront_SoilWatVol.txt".format(n_days, run_number)
relationship_df = pd.read_table(relationship_filename, header=0, index_col=0)
#relationship_df.head()

precipitation_column_names = ["timestep", "day", "hour", "precip"]
precipitation_filename = working_dir+watershed_nick+"__2017_0401_1228_precipitation.csv"
precipitation_df = pd.read_csv(precipitation_filename, header=0, index_col=[1,2], names=precipitation_column_names)
precipitation_df = precipitation_df.loc[("2017-05-01", "00:00:00"):("2017-10-26", "01:00:00")]
days = precipitation_df.day
hours = precipitation_df['hour']
precipitation_df = precipitation_df.set_index("timestep", drop=False, append=False, 
                                              inplace=False, verify_integrity=False)
#print(pd.MultiIndex.from_frame(precipitation_df))
print(precipitation_df.head())
#print(precipitation_df.shape)

observation_filename = working_dir+watershed_nick+"_hourly_obs_2017_0501_1026.csv"
observation_column_names = ["timestep","day","hour","streamflow_cfs","streamflow_ms"]
observation_df = pd.read_csv(observation_filename, header=0, index_col=[1,2], 
                             usecols=[0,1,2,3,4], names=observation_column_names)
#print(observation_df.head())
#print(observation_df.shape)


C:\Users\feder\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.


AttributeError: 'DataFrame' object has no attribute 'day'

In [107]:
def interpolate_from_storageRatio(targetStorageRatio, column):
    if targetStorageRatio < min(relationship_df["norm_SoilWatVol"]):
        return 0
    upperStorageRatio = min(relationship_df[relationship_df["norm_SoilWatVol"] > targetStorageRatio]["norm_SoilWatVol"])
    lowerStorageRatio = max(relationship_df[relationship_df["norm_SoilWatVol"] < targetStorageRatio]["norm_SoilWatVol"])
    #print(upperStorageRatio, lowerStorageRatio)
    upperRatio = relationship_df.loc[relationship_df["norm_SoilWatVol"] == upperStorageRatio, column].iloc[0]
    lowerRatio = relationship_df.loc[relationship_df["norm_SoilWatVol"] == lowerStorageRatio, column].iloc[0]
    if upperRatio == lowerRatio:
        return upperRatio
    
    targetRatio = lowerRatio + (targetStorageRatio-lowerStorageRatio)*(upperRatio-lowerRatio)/(upperStorageRatio-lowerStorageRatio)
    return targetRatio



def plotFlow(times, days, hours, precip, streamFlow, streamFlow_observed):    
    fig,ax = plt.subplots(figsize=(14,5.5))
    ax.set_title(watershed_name + ' ', fontsize=20)
    #ax.set_ylabel(r'$\frac{A_{sat}}{A_{tot}}$', fontsize=22, labelpad=22, rotation='horizontal')
    ax.set_ylabel(r'Streamflow per unit area [m/h]', fontsize=16, labelpad=16, rotation='vertical')
    ax.set_xlabel("Time", fontsize=16, labelpad=10, rotation='horizontal')
    spaced_yticks = [min(streamFlow),
                     min(streamFlow)+(max(streamFlow)-min(streamFlow))*0.25,
                     min(streamFlow)+(max(streamFlow)-min(streamFlow))*0.5,
                     min(streamFlow)+(max(streamFlow)-min(streamFlow))*0.75,
                     max(streamFlow)]
    ax.set_yticks(spaced_yticks)
    ax.set_yticklabels(['{0:.2f}'.format(spaced_yticks[0]),
                        '{0:.2f}'.format(spaced_yticks[1]),
                        '{0:.2f}'.format(spaced_yticks[2]),
                        '{0:.2f}'.format(spaced_yticks[3]),
                        '{0:.2f}'.format(spaced_yticks[-1])], 
                       fontsize=13)
    date_time_obj = pd.to_datetime(days)
    dates = date_time_obj.apply(lambda x: x.strftime("%d-%b"))
    first_days_of_month_1 = []
    first_days_of_month_1 = [first for first in dates if '01-' in first]
    first_days_of_month= []
    first_days_of_month = [first for first in first_days_of_month_1 if first not in first_days_of_month]
    print(first_days_of_month)
    first_days_of_month_indices = [0, 31*24, 61*24]
    #print(first_days_of_month_indices)
    ax.set_xticks(first_days_of_month_indices)
    ax.set_xticklabels(first_days_of_month)
    ax.plot(times, streamFlow, color='r', linewidth=0.7)
    ax.plot(times, streamFlow_observed, color='k', linewidth=0.7)
    
    ax2 = ax.twinx()
    ax2.bar(times, precip, 0.2, color='dodgerblue')
    #print(precip.head())
    ax2.invert_yaxis()
    ax2.set_ylabel(r'Precipitation [mm]', fontsize=16, labelpad=16, rotation='vertical')
    #y2_ticks = np.linspace(0, max(precip), max(precip))
    #y2_ticklabels = [str(i) for i in y2_ticks]
    #ax2.set_yticks(-1 * y2_ticks)
    #ax2.set_yticklabels(y2_ticklabels)
    fig.show()

    
    
def plotDeficitRatio(times, days, hours, precip, deficitRatio):
    deficitRatio = deficitRatio*100
    fig,ax = plt.subplots(figsize=(14, 5.5))
    ax.set_title(watershed_name + ' ', fontsize=20)
    #ax.set_ylabel(r'$\frac{A_{sat}}{A_{tot}}$', fontsize=22, labelpad=22, rotation='horizontal')
    ax.set_ylabel(r'Deficit ratio', fontsize=16, labelpad=16, rotation='vertical')
    ax.set_xlabel("Time", fontsize=16, labelpad=10, rotation='horizontal')
    spaced_yticks = [min(deficitRatio),
                     min(deficitRatio)+(max(deficitRatio)-min(deficitRatio))*0.25,
                     min(deficitRatio)+(max(deficitRatio)-min(deficitRatio))*0.5,
                     min(deficitRatio)+(max(deficitRatio)-min(deficitRatio))*0.75,
                     max(deficitRatio)]
    ax.set_yticks(spaced_yticks)
    ax.set_yticklabels(['{0:.2f}'.format(spaced_yticks[0]),
                        '{0:.2f}'.format(spaced_yticks[1]),
                        '{0:.2f}'.format(spaced_yticks[2]),
                        '{0:.2f}'.format(spaced_yticks[3]),
                        '{0:.2f}'.format(spaced_yticks[-1])], 
                       fontsize=13)
    ax.plot(times, deficitRatio, color='r', linewidth=0.7)
    
    ax2 = ax.twinx()
    ax2.bar(times, precip, 0.2, color='dodgerblue')
    #print(precip.head())
    ax2.invert_yaxis()
    ax2.set_ylabel(r'Precipitation [mm]', fontsize=16, labelpad=16, rotation='vertical')
    #y2_ticks = np.linspace(0, max(precip), max(precip)+1)
    #y2_ticklabels = [str(i) for i in y2_ticks]
    #ax2.set_yticks(-1 * y2_ticks)
    #ax2.set_yticklabels(y2_ticklabels)
    fig.show()


    
def plotAreaRatio(times, days, hours, precip, areaRatio):
    areaRatio = areaRatio*100
    fig,ax = plt.subplots(figsize=(14,5.5))
    ax.set_title(watershed_name + ' ', fontsize=20)
    #ax.set_ylabel(r'$\frac{A_{sat}}{A_{tot}}$', fontsize=22, labelpad=22, rotation='horizontal')
    ax.set_ylabel(r'Area ratio', fontsize=16, labelpad=16, rotation='vertical')
    ax.set_xlabel("Time", fontsize=16, labelpad=10, rotation='horizontal')
    spaced_yticks = [min(areaRatio),
                     min(areaRatio)+(max(areaRatio)-min(areaRatio))*0.25,
                     min(areaRatio)+(max(areaRatio)-min(areaRatio))*0.5,
                     min(areaRatio)+(max(areaRatio)-min(areaRatio))*0.75,
                     max(areaRatio)]
    ax.set_yticks(spaced_yticks)
    ax.set_yticklabels(['{0:.2f}'.format(spaced_yticks[0]),
                        '{0:.2f}'.format(spaced_yticks[1]),
                        '{0:.2f}'.format(spaced_yticks[2]),
                        '{0:.2f}'.format(spaced_yticks[3]),
                        '{0:.2f}'.format(spaced_yticks[-1])], 
                       fontsize=13)
    ax.plot(times, areaRatio, color='r', linewidth=0.7)
    
    ax2 = ax.twinx()
    ax2.bar(times, precip, 0.2, color='dodgerblue')
    #print(precip.head())
    ax2.invert_yaxis()
    ax2.set_ylabel(r'Precipitation [mm]', fontsize=16, labelpad=16, rotation='vertical')
    #y2_ticks = np.linspace(0, max(precip), max(precip)+1)
    #y2_ticklabels = [str(i) for i in y2_ticks]
    #ax2.set_yticks(-1 * y2_ticks)
    #ax2.set_yticklabels(y2_ticklabels)
    fig.show()


In [130]:
output_Dict = {}
initialStorageRatio = 0.80
initialStorageVol = maxStorage * initialStorageRatio
initialDeficitRatio = 1 - initialStorageRatio
initialDeficitVol = maxStorage * initialDeficitRatio
initialAreaRatio = interpolate_from_storageRatio(initialStorageRatio, "AreaRatio")
print("initialAreaRatio is {}".format(initialAreaRatio))
initialAsat = initialAreaRatio * A
#initialFront = interpolate_from_storageRatio(initialStorageRatio, "Saturated_Front")

currentAsat = initialAsat
currentStorageVol = initialStorageVol
currentStorageRatio = initialStorageRatio
#currentFront = initialFront

days_hours = precipitation_df.index
initial_day_hour, final_day_hour = ("2017-05-01", "00:00:00"), ("2017-07-07", "10:00:00")
initial_timestep = precipitation_df.timestep.loc[initial_day_hour]
final_timestep = precipitation_df.timestep.loc[final_day_hour]
print(precipitation_df.head())
multiindex = pd.MultiIndex.from_frame(precipitation_df)
multiindex.names = ['day','hour']

multiindex.get_level_values(0)

initialAreaRatio is 0.08808572622998122
                     timestep  precip
day        hour                      
2017-05-01 00:00:00       721   0.000
           01:00:00       722   0.400
           02:00:00       723   0.075
           03:00:00       724   0.000
           04:00:00       725   0.000


Int64Index([ 721,  722,  723,  724,  725,  726,  727,  728,  729,  730,
            ...
            4985, 4986, 4987, 4988, 4989, 4990, 4991, 4992, 4993, 4994],
           dtype='int64', name='day', length=4274)

In [116]:
for day_hour in multiindex:
    print("Running timestep {}".format(day_hour))
    previousAsat = currentAsat
    previousStorageVol = currentStorageVol
    previousStorageRatio = currentStorageRatio
    #previousFront = currentFront
    
    timestep = precipitation_df.timestep.loc[day_hour]
    print(timestep)
    #hour = precipitation_df['hour'].loc[timestep]
    P = precipitation_df['precip'].loc[day_hour]/1000
    totalP = P * A
    Qrunoff = P * previousAsat
    Qsubsurface = b * A * previousStorageRatio**n
    norm_Qrunoff = Qrunoff / A
    norm_Qsubsurface = Qsubsurface / A
    streamFlow = Qrunoff + Qsubsurface
    norm_streamflow = norm_Qrunoff + norm_Qsubsurface
    deltaStorageVol = totalP - ET - Qrunoff - Qsubsurface
    currentStorageVol = previousStorageVol + deltaStorageVol
    currentStorageRatio = currentStorageVol/maxStorage
    currentAreaRatio = interpolate_from_storageRatio(currentStorageRatio, "AreaRatio")
    currentAsat = currentAreaRatio * A
    #currentFront = interpolate_from_storageRatio(currentStorageRatio, "Saturated_Front")
    output_Dict[timestep] = [day_hour, 
                             P*1000, Qrunoff, Qsubsurface, streamFlow, norm_streamflow, 
                             currentStorageRatio, currentAreaRatio]  

output_df = pd.DataFrame.from_dict(output_Dict,
                                   orient = 'index',
                                   columns = ["day_hour", 
                                              "P", "Qse","Qgw","streamFlow", "norm_streamFlow", 
                                              "storageRatio", "areaRatio"])
times = output_df.index
days_hours = output_df["day_hour"]
precip = output_df["P"]
Qse = output_df["Qse"]
Qgw = output_df["Qgw"]
streamFlow = output_df["streamFlow"]
norm_streamFlow = output_df["norm_streamFlow"]
storageRatio = output_df["storageRatio"]
areaRatio = output_df["areaRatio"]
observed_streamFlow = observation_df["streamflow_ms"][initial_timestep:final_timestep+1]*3600
#print(observed_streamFlow.head())
plotFlow(times, days_hours, precip, norm_streamFlow, observed_streamFlow)
plotDeficitRatio(times, days_hours, precip, 1-storageRatio)
plotAreaRatio(times, days_hours, precip, areaRatio)

initialAreaRatio is 0.08808572622998122
Running timestep (721, 0.0)


IndexingError: Too many indexers

In [ ]:
btry = observed_streamFlow.iloc[1]/3600/(0.89)**0.6
print(observed_streamFlow.iloc[1])
print(btry)

In [ ]:
date_time_obj = pd.to_datetime(days)
dates = date_time_obj.apply(lambda x: x.strftime("%d-%b"))
print(dates)
first_days_of_month = [first for first in dates if '01' in first]
first_days_of_month